In [1]:
pip install requests numpy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import warnings
warnings.filterwarnings("ignore")

import requests
import numpy as np
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm.notebook import tqdm
import random

In [6]:
UserAgent().chrome

'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Mobile Safari/537.36'

In [7]:
#Собирает ссылки на статьи с главной страницы Habr
def get_article_links(pages=10): #pages - количество страниц для парсинга
    base_url = "https://habr.com/ru/all/page{}/" #шаблон URL для разных страниц {} - номер страницы
    article_links = []
    
    for page in tqdm(range(1, pages + 1), desc="Сбор ссылок"): #tqdm для визуализации
        url = base_url.format(page)
        response = requests.get(url, headers={'User-Agent': UserAgent().chrome}) #useragent имитираует браузер
        
        if response.status_code != 200: #проверка успешности запроса(код 200)
            print(f"Ошибка на странице {page}: {response.status_code}")
            continue
                
        soup = BeautifulSoup(response.content, 'html.parser')
            
        # Поиск ссылок
        articles = soup.find_all('article', class_='tm-articles-list__item')
            
        for article in articles:
            link_tag = article.find('a', class_='tm-title__link')
            if link_tag and 'href' in link_tag.attrs:
                full_link = "https://habr.com" + link_tag['href']
                article_links.append(full_link)
            
        # От блокировки
        time.sleep(random.uniform(1, 3))
    
    # Удалить дубликаты
    article_links = list(set(article_links))
    print(f"Собрано {len(article_links)} уникальных ссылок на статьи")
    return article_links

In [8]:
# Собирает данные из отдельной статьи
def parse_article(article_url):
    # Загрузка и проверка страницы
    response = requests.get(article_url, headers={'User-Agent': UserAgent().chrome})
    if response.status_code != 200:
        return None
        
    soup = BeautifulSoup(response.content, 'html.parser') # Парсинг html
    
    data = {}
    
    title_elem = soup.find('h1', class_='tm-title tm-title_h1')
    data['title'] = title_elem.text.strip() if title_elem else ""
    
    author_elem = soup.find('a', class_='tm-user-info__username')
    data['author'] = author_elem.text.strip() if author_elem else ""
    
    date_elem = soup.find('time')
    data['publish_date'] = date_elem['datetime'] if date_elem and 'datetime' in date_elem.attrs else ""
    
    hubs = []
    hubs_elems = soup.find_all('a', class_='tm-publication-hub__link')
    for hub in hubs_elems:
        hubs.append(hub.text.strip())
    data['hubs'] = hubs
    
    article_elem = soup.find('div', class_='tm-article-body')
    data['text'] = article_elem.text.strip() if article_elem else ""
    
    views_elem = soup.find('span', class_='tm-icon-counter__value')
    data['views'] = int(views_elem.text.strip()) if views_elem else 0
    
    comments_elem = soup.find('span', class_='tm-article-comments-counter-link__value')
    if comments_elem:
        comments_text = comments_elem.text.strip()
        data['comments'] = int(comments_text) if comments_text.isdigit() else 0
    else:
        data['comments'] = 0
    
    rating_elem = soup.find('span', class_='tm-votes-meter__value')
    data['rating'] = int(rating_elem.text.strip()) if rating_elem else 0
    
    reading_time_elem = soup.find('span', class_='tm-article-reading-time__label')
    data['reading_time'] = reading_time_elem.text.strip() if reading_time_elem else ""
    
    data['url'] = article_url
    
    # Дополнительные метрики из текста
    if data['text']:
        data['text_length'] = len(data['text'])
        data['word_count'] = len(data['text'].split())
    else:
        data['text_length'] = 0
        data['word_count'] = 0
        
    return data

In [9]:
pip show ipywidgets

Name: ipywidgets
Version: 8.1.8
Summary: Jupyter interactive widgets
Home-page: http://jupyter.org
Author: Jupyter Development Team
Author-email: jupyter@googlegroups.com
License: BSD 3-Clause License
Location: c:\Users\anmrt\Desktop\Useful shit\nlp_labs\venv\Lib\site-packages
Requires: comm, ipython, jupyterlab_widgets, traitlets, widgetsnbextension
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [13]:
def main():
    print("Сбор ссылок")
    article_links = get_article_links(pages=20)
    
    # Сохранить ссылки в файл
    with open('habr_article_links.txt', 'w', encoding='utf-8') as f:
        for link in article_links:
            f.write(link + '\n')
    
    print("\nПарсинг статей")
    articles_data = []
    
    for i, link in enumerate(tqdm(article_links, desc="Парсинг статей")):
        article_data = parse_article(link)
        if article_data:
            articles_data.append(article_data)
        
        time.sleep(random.uniform(2, 4)) # Задержка от блокировки
        
        # Сохранение прогресса каждые 50 статей
        if i % 50 == 0 and i > 0:
            temp_df = pd.DataFrame(articles_data)
            temp_df.to_csv(f'habr_articles_progress_{i}.csv', index=False, encoding='utf-8')
            print(f"Сохранен прогресс после {i} статей")
    
    print("\nСохранение данных")
    df = pd.DataFrame(articles_data)
    
    df.to_csv('habr_articles_dataset.csv', index=False, encoding='utf-8')
    
    print(f"Всего собрано статей: {len(df)}")
    print(f"Столбцы в датасете: {list(df.columns)}")
    
    # Выводим базовую статистику
    print("\nБазовая статистика:")
    print(f"Среднее количество просмотров: {df['views'].mean():.0f}")
    print(f"Среднее количество комментариев: {df['comments'].mean():.0f}")
    print(f"Средняя длина текста: {df['text_length'].mean():.0f} символов")
    
    return df

if __name__ == "__main__":
    df = main()

Сбор ссылок


Сбор ссылок:   0%|          | 0/20 [00:00<?, ?it/s]

Собрано 397 уникальных ссылок на статьи

Парсинг статей


Парсинг статей:   0%|          | 0/397 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: '1.7K'